## 数据集处理
这里原本是把字符改为不重复的数字，那为什么不去改成一串唯一的特征向量，各个字符之间有天然的相关性联系

In [ ]:
# 随机采样
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    """
    随机采样小批量子序列，用于字符级 RNN 训练。
    X: 当前字符序列
    Y: X 整体右移一位（下一个字符）
    """

    # 1. 计算总样本数（整段序列能切出多少个长度为 num_steps 的样本）
    # 减 1：因为标签 Y 需要整体右移一位，最后一个位置不能当起点
    # 至少要 num_steps + 1 个字符，才能构成 1 个 (X, Y) 样本
    num_examples = (len(corpus_indices) - 1) // num_steps

    # 2. 计算每个 epoch 能有多少个完整 batch
    # 多出来凑不满一个 batch 的样本直接丢弃
    epoch_size = num_examples // batch_size

    # 3. 样本编号列表 [0, 1, 2, ..., num_examples-1]，然后打乱，实现随机抽样
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 4. 给定起点 pos，返回一段长度为 num_steps 的子序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    # 5. 逐个 batch 产生数据
    for i in range(epoch_size):
        # 当前 batch 对应的样本编号区间起点（按 batch_size 递增）
        i = i * batch_size

        # 当前 batch 里要用到的样本“编号”（已经被打乱过）
        batch_indices = example_indices[i: i + batch_size]

        # 6. 构造当前 batch 的 X 和 Y
        # 每个样本的起点是 j * num_steps
        # X: 从 j * num_steps 开始的 num_steps 长度
        X = [_data(j * num_steps) for j in batch_indices]
        # Y: 从 j * num_steps + 1 开始，整体右移一位，作为“下一个字符”
        Y = [_data(j * num_steps + 1) for j in batch_indices]

        # 7. 转成 MXNet NDArray，放到指定设备（CPU/GPU），并 yield 出一个 batch
        yield nd.array(X, ctx), nd.array(Y, ctx)


In [4]:
from mxnet import nd
import random
import zipfile
# 相邻采样（顺序采样）
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].reshape((
        batch_size, batch_len))
    print(indices)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
        
my_seq = list(range(30))
X, Y = data_iter_consecutive(my_seq, batch_size=2, num_steps=6)

#indices =
#[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
# [15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29.]]
#<NDArray 2x15 @cpu(0)>



[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29.]]
<NDArray 2x15 @cpu(0)>
